In [ ]:
%pip install requests beautifulsoup4 tqdm
%pip install selenium webdriver-manager tqdm

In [5]:
import os
import time
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager


# ================= SETTINGS =================

SAVE_DIR = "Data"
BASE_URL = "https://laws.moj.gov.sa/ar/legislations-regulations"

MAX_PAGES = 300      # عدّل لو الصفحات أكتر
WAIT_TIME = 6        # وقت الانتظار بين الصفحات

# ============================================


os.makedirs(SAVE_DIR, exist_ok=True)


# Chrome Options
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")


driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

wait = WebDriverWait(driver, 40)


# Requests session
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0"
})


def download_file(url, filename):

    r = session.get(url, stream=True)
    r.raise_for_status()

    path = os.path.join(SAVE_DIR, filename)

    with open(path, "wb") as f:
        for chunk in r.iter_content(1024):
            if chunk:
                f.write(chunk)


# Open site
driver.get(BASE_URL)
time.sleep(8)


count = 0

print("🚀 Start Downloading...\n")


for page in range(1, MAX_PAGES + 1):

    print(f"📄 Page {page}")


    # Get PDF buttons
    try:
        buttons = wait.until(
            EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, "div.deatils.clickable")
            )
        )
    except:
        print("⛔ No items found. Stop.")
        break


    print(f"   ➜ Found {len(buttons)} files")


    # Click every PDF
    for btn in buttons:

        try:
            driver.execute_script(
                "arguments[0].scrollIntoView({block:'center'});",
                btn
            )

            time.sleep(1)

            btn.click()


            # Wait new tab
            wait.until(EC.number_of_windows_to_be(2))

            tabs = driver.window_handles


            # Switch to PDF tab
            driver.switch_to.window(tabs[1])

            time.sleep(3)

            pdf_url = driver.current_url


            count += 1

            print(f"⬇️ Downloading ({count}): {pdf_url}")


            download_file(pdf_url, f"law_{count}.pdf")


            # Close PDF tab
            driver.close()
            driver.switch_to.window(tabs[0])

            time.sleep(2)

        except Exception as e:
            print("⚠️ Skip one file:", e)

            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])


    # Go to Next Page
    try:
        next_btn = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, 'button[aria-label="Go to next page"]')
            )
        )

        driver.execute_script(
            "arguments[0].scrollIntoView({block:'center'});",
            next_btn
        )

        time.sleep(1)

        next_btn.click()

        print("➡️ Next page\n")

        time.sleep(WAIT_TIME)

    except:
        print("⛔ No more pages. Done.")
        break


driver.quit()

print("\n🎉 Finished!")
print(f"📁 Total downloaded: {count}")
print(f"📂 Saved in: {SAVE_DIR}")


🚀 Start Downloading...

📄 Page 1
   ➜ Found 9 files
⬇️ Downloading (1): https://laws-gateway.moj.gov.sa/apis/legislations/v1/document/download?Document=uAddqnglQn_ReHgSHBze8us_otYXOiPOH2J0wCfACN7JEwHwhho2dF9sBiznq070Bbgi36E7aRahUi9XHRad3g
⬇️ Downloading (2): https://laws-gateway.moj.gov.sa/apis/legislations/v1/document/download?Document=uAddqnglQn_ReHgSHBze8h-XvDiep6qlmNXsT1Q2LLrpFR13WNM2wFR7tV8mr4yUXqKU1zyByBdOw4-9-zhYRQ
⬇️ Downloading (3): https://laws-gateway.moj.gov.sa/apis/legislations/v1/document/download?Document=uAddqnglQn_ReHgSHBze8qTXeglyT8AndmZbPioVlQmrLPwvU4ymy3YvDLsIQzE67O2ZoOgLkhzNs_k3mGRbyg
⬇️ Downloading (4): https://laws-gateway.moj.gov.sa/apis/legislations/v1/document/download?Document=uAddqnglQn_ReHgSHBze8oEOjJVrRBXYypGcn-eZWBUFO-rGFM6AlxLsFaB7mVI2Xv4BtlR4R0mFck80fHPjgg
⬇️ Downloading (5): https://laws-gateway.moj.gov.sa/apis/legislations/v1/document/download?Document=uAddqnglQn_ReHgSHBze8jgY6_aPPw1t5loMwZXrxS8SU76b32l_7rtS_x47Kj0lk1CqVVJUUMaDEBrzMZO5hg
⬇️ Downloadin